In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/patil182/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("spotify_millsongdata.csv")

In [4]:
df = df.sample(20000).drop("link", axis = 1).reset_index(drop=True)

In [5]:
# Preprocessing text data
df['text'] = df['text'].str.lower().replace(r'^\w\s', '').replace(r'\n', ' ', regex = True)

In [6]:

from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

In [7]:
def tokenizor(txt):
    token = nltk.word_tokenize(txt)
    similar_words = [stemmer.stem(w) for w in token]
    return " ".join(similar_words)
    

In [8]:
df['text'].apply(lambda x: tokenizor(x))

0        ( with bob marley ) [ bob marley ] ( lauryn - ...
1        i wake up in a differ daylight guess i have n'...
2        last night i dream i wa look down on the plane...
3        in the eye of the world he 's onli an ordinari...
4        pretti smile love face and a warm breez now i ...
                               ...                        
19995    give someon some piec of your mind . shine it ...
19996    ai n't the pictur enough , whi do you go throu...
19997    take a look insid i 've got noth to hide , oh ...
19998    time is a thing we must accept the unexpect i ...
19999    everytim we say goodby , i die a littl , every...
Name: text, Length: 20000, dtype: object

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfid = TfidfVectorizer(analyzer='word', stop_words='english')

In [10]:
matrix = tfid.fit_transform(df['text'])

In [11]:
similarity = cosine_similarity(matrix)

In [12]:
def recommendation_function(song_name):
    idx = df[df['song'] == song_name].index[0]
    distance = sorted(list(enumerate(similarity[idx])), reverse=True, key = lambda x:x[1])
    song = []
    for s_id in distance[1:5]:
        song.append(df.iloc[s_id[0]].song)
    return song

In [13]:
df

,artist,song,text
0,Lauryn Hill,Turn Your Lights Down Low,(with bob marley) \r \r [bob marley] \r (l...
1,Roxette,You Turn Me On,i wake up in a different daylight \r guess i ...
2,Dan Fogelberg,Holy Road,last night i dreamed i was looking down \r on...
3,Reba Mcentire,He's Only Everything,in the eyes of the world he's only an ordinary...
4,Chicago,No Tell Lover,pretty smile lovely face and a warm breeze now...
...,...,...,...
19995,Quicksand,Too Official,give someone some piece of your mind. \r shin...
19996,Michael Jackson,Privacy,"ain't the pictures enough, why do you go throu..."
19997,Imagine Dragons,Leave Me,take a look inside \r i've got nothing to hid...
19998,Death,Destiny,time is a thing we must accept \r the unexpec...


In [14]:
recommendation_function("No Tell Lover")

['For My Lover',
 'Lover Come Back To Me',
 "I Don't Want To Be Your Lover",
 'Drive You Home']

In [15]:
import pickle
pickle.dump(similarity, open("similarity", "wb"))
pickle.dump(df, open("df", "wb"))